In [2]:
from frust.transformers import transformer_ts
from frust.stepper import Stepper
from frust.embedder import embed_ts

In [ ]:
def run_ts1(
    ligand_smiles_list: list[str],
    ts_guess_xyz: str,
    *,
    n_confs: int | None = None,
    n_cores: int = 4,
    debug: bool = False,
    top_n: int = 5,
    out_dir: str = None,
    output_parquet: str | None = None,
    save_output_dir: bool = True,
):
    # 1) build TS guesses
    ts_structs = {}
    for smi in ligand_smiles_list:
        ts_structs.update({
            name: (mol, idxs, smi)
            for name, (mol, idxs) in transformer_ts(
                ligand_smiles=smi, ts_guess_struct=ts_guess_xyz
            ).items()
        })

    # 2) embed
    embedded = embed_ts(ts_structs, n_confs=n_confs, optimize=not debug)

    # 3) xTB cascade
    step = Stepper(
    ligand_smiles_list,
    n_cores=n_cores,
    debug=debug,
    output_base=out_dir,
    save_output_dir=save_output_dir,
    )
    df0 = step.build_initial_df(embedded)
    df1 = step.xtb(df0, options={"gfnff": None, "opt": None}, constraint=True)
    df2 = step.xtb(df1, options={"gfn": 2})

    # ---------- identical filtering to notebook ----------
    df2_filt = (
        df2.sort_values(["ligand_name", "rpos", "xtb-gfn-electronic_energy"])
           .groupby(["ligand_name", "rpos"])
           .head(top_n)
    )

    df3 = step.xtb(df2_filt, options={"gfn": 2, "opt": None}, constraint=True)

    df3_fin = (
        df3.sort_values(["ligand_name", "rpos", "xtb-gfn-opt-electronic_energy"])
           .groupby(["ligand_name", "rpos"])
           .head(1)
    )

    return df3_fin

    if output_parquet:
        df3_fin.to_parquet(output_parquet)
    return df3_fin

In [20]:
df = run_ts1(["CN1C=CC=C1"], ts_guess_xyz=f"../structures/ts1.xyz", n_confs=1, debug=False, save_output_dir=False)

Embedded 1 conformers on atom 44
Embedded 1 conformers on atom 45
2025-06-11 14:13:29 INFO  frust.stepper: [xtb-gfnff-opt] row 0 (TS(1-methylpyrrole_rpos(2)))…
2025-06-11 14:13:29 INFO  frust.stepper: [xtb-gfnff-opt] row 1 (TS(1-methylpyrrole_rpos(3)))…
2025-06-11 14:13:29 INFO  frust.stepper: [xtb-gfn] row 0 (TS(1-methylpyrrole_rpos(2)))…
2025-06-11 14:13:29 INFO  frust.stepper: [xtb-gfn] row 1 (TS(1-methylpyrrole_rpos(3)))…
2025-06-11 14:13:29 INFO  frust.stepper: [xtb-gfn-opt] row 0 (TS(1-methylpyrrole_rpos(2)))…
2025-06-11 14:13:31 INFO  frust.stepper: [xtb-gfn-opt] row 1 (TS(1-methylpyrrole_rpos(3)))…


In [21]:
df

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff,xtb-gfnff-opt-electronic_energy,xtb-gfnff-opt-normal_termination,xtb-gfnff-opt-opt_coords,xtb-gfn-electronic_energy,xtb-gfn-normal_termination,xtb-gfn-opt-electronic_energy,xtb-gfn-opt-normal_termination,xtb-gfn-opt-opt_coords
0,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.2225199327853987, -1.585347835373733, -0.6...",4555.073125,-8.967520,True,"[[0.1406627016508, -1.51748358124332, -0.54095...",-65.684801,True,-65.702001,True,"[[0.17552530507115, -1.50462408622571, -0.5520..."
1,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.5057052003345333, -1.6799352838707575, -0....",5064.591841,-8.955958,True,"[[0.41798922062639, -1.62135511985775, -0.3748...",-65.679951,True,-65.697268,True,"[[0.4567503039369, -1.62589932682468, -0.33511..."
